In [1]:
import qlib
import pandas as pd
from qlib.constant import REG_CN
from qlib.utils import exists_qlib_data, init_instance_by_config
from qlib.workflow import R
from qlib.workflow.record_temp import SignalRecord, PortAnaRecord
from qlib.utils import flatten_dict
from qlib.data.filter import NameDFilter


import universal as up
from universal import tools, algos
from universal.algos import *

from lilab.qlib.model.meanvar_signal import SimpleMeanVar
from lilab.qlib.utils.tools import get_return_dataframe
from lilab.qlib.strategy.meanvar_strategy import MeanVarStrategy

from upadapter import UPAdapter

import warnings
warnings.filterwarnings('ignore')

provider_uri = "~/.qlib/qlib_data/cn_data"  # target_dir
qlib.init(provider_uri=provider_uri, region=REG_CN)

/home/linq/anaconda3/envs/qlib39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[3041700:MainThread](2024-11-27 15:37:12,071) INFO - qlib.Initialization - [config.py:416] - default_conf: client.
[3041700:MainThread](2024-11-27 15:37:12,074) INFO - qlib.Initialization - [__init__.py:74] - qlib successfully initialized based on client settings.
[3041700:MainThread](2024-11-27 15:37:12,075) INFO - qlib.Initialization - [__init__.py:76] - data_path={'__DEFAULT_FREQ': PosixPath('/home/linq/.qlib/qlib_data/cn_data')}


In [2]:
algo = OLMAR()

In [3]:
up_adapter = UPAdapter('csi300_ext', '2024-01-01', '2024-07-01', algo)

[3041700:MainThread](2024-11-27 15:37:12,818) INFO - qlib.timer - [log.py:127] - Time cost: 0.729s | Loading data Done
[3041700:MainThread](2024-11-27 15:37:12,822) INFO - qlib.timer - [log.py:127] - Time cost: 0.000s | Fillna Done
[3041700:MainThread](2024-11-27 15:37:12,823) INFO - qlib.timer - [log.py:127] - Time cost: 0.002s | fit & process data Done
[3041700:MainThread](2024-11-27 15:37:12,824) INFO - qlib.timer - [log.py:127] - Time cost: 0.735s | Init data Done


In [4]:
df = up_adapter.data
df

instrument,SH600000,SH600009,SH600010,SH600011,SH600015,SH600016,SH600018,SH600019,SH600023,SH600025,...,SZ300760,SZ300763,SZ300782,SZ300832,SZ300896,SZ300919,SZ300957,SZ300979,SZ300999,SZ301269
datetime,,,,,,,,,,,,,,,,,,,,,
2024-01-02,1.253317,2.143291,0.908943,1.520562,1.436759,1.381437,1.299861,1.183318,1.002955,3.362033,...,4.427734,8.064856,13.709311,3.432389,1.543640,0.571872,0.406342,0.560835,0.589240,1.401820
2024-01-03,1.260913,2.141293,0.915211,1.583285,1.454403,1.381437,1.323352,1.208914,1.027517,3.388656,...,4.372363,8.033031,13.297001,3.394627,1.541049,0.572938,0.405968,0.548499,0.589240,1.380459
2024-01-04,1.257115,2.109323,0.902674,1.600391,1.461965,1.381437,1.325962,1.210883,1.031611,3.384853,...,4.312164,7.856221,13.120739,3.343693,1.491607,0.567491,0.398870,0.548288,0.578830,1.367909
2024-01-05,1.268509,2.094671,0.896406,1.587086,1.487171,1.399808,1.331183,1.216790,1.033658,3.362033,...,4.280179,7.672339,13.039308,3.255874,1.464836,0.556480,0.396317,0.545652,0.569676,1.345881
2024-01-08,1.251418,2.057373,0.883868,1.606093,1.484650,1.381437,1.328573,1.210883,1.023423,3.327805,...,4.210927,7.598080,12.517735,3.282220,1.528095,0.546416,0.392145,0.545652,0.558010,1.332263
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-25,1.557152,2.132634,0.865063,1.758149,1.687674,1.359393,1.516566,1.336278,1.424605,4.094325,...,4.445775,5.174840,8.145043,3.012346,1.306331,0.526385,0.292462,0.701187,0.498356,1.032998
2024-06-26,1.559051,2.152615,0.883868,1.750547,1.682325,1.374089,1.513877,1.338306,1.410277,4.090455,...,4.473068,5.265917,8.396153,3.034146,1.353934,0.546879,0.306823,0.698182,0.506641,1.058682
2024-06-27,1.560950,2.131302,0.871331,1.748646,1.698372,1.381437,1.508499,1.330195,1.404137,4.098194,...,4.465487,5.095592,8.171911,3.044823,1.318270,0.533329,0.311674,0.680472,0.499977,1.040757


In [5]:
# tools.dataset('nyse_o')

In [6]:
up_adapter.predict()

Summary:
    Profit factor: 0.97
    Sharpe ratio: 0.02 ± 1.42
    Ulcer index: 0.07
    Information ratio (wrt benchmark): -0.09
    Benchmark sharpe: 0.21 ± 1.44
    Appraisal ratio (wrt benchmark): -0.14 ± 1.43
    Beta / Alpha: 1.46 / -4.537%
    Annualized return: 0.68%
    Annualized volatility: 40.89%
    Longest drawdown: 49 days
    Max drawdown: 27.02%
    Winning days: 47.4%
    Annual turnover: 308.7
        


In [7]:
df = up_adapter.get_weight()
df

,SH600000,SH600009,SH600010,SH600011,SH600015,SH600016,SH600018,SH600019,SH600023,SH600025,...,SZ300760,SZ300763,SZ300782,SZ300832,SZ300896,SZ300919,SZ300957,SZ300979,SZ300999,SZ301269
datetime,,,,,,,,,,,,,,,,,,,,,
2024-01-02,0.003215,0.003215,0.003215,0.003215,0.003215,0.003215,0.003215,0.003215,0.003215,0.003215,...,0.003215,0.003215,0.003215,0.003215,0.003215,0.003215,0.003215,0.003215,0.003215,0.003215
2024-01-03,0.003215,0.003215,0.003215,0.003215,0.003215,0.003215,0.003215,0.003215,0.003215,0.003215,...,0.003215,0.003215,0.003215,0.003215,0.003215,0.003215,0.003215,0.003215,0.003215,0.003215
2024-01-04,0.000000,0.000000,0.000000,0.241964,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2024-01-05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2024-01-08,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-25,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2024-06-26,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2024-06-27,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [8]:
up_adapter.get_qlib_weight()

weight
date       stock             
2024-01-02 SH600000  0.003215
           SH600009  0.003215
           SH600010  0.003215
           SH600011  0.003215
           SH600015  0.003215
...                       ...
2024-06-26 SZ300496  0.986147
2024-06-27 SH688223  0.825902
           SH688303  0.174098
2024-06-28 SH688223  1.000000
2024-07-01 SZ300059  1.000000

[873 rows x 1 columns]